Link to GitHub repository [here](https://github.com/jesp9435/ComSocSci)

Group member contributions: Both group members contributed equally to the parts of the assignment. We have worked collaboratively on all parts. 

# Part 1: Web-scraping

In total we got a total of 1484 unique researchers. 

First we inspected the HTML page, and used the tool to select an element on a page. Using this, it was possible to see a pattern on the website, e.g. that all the conferences were nested under a single class. We thought looking for this would yield the max amount of authors. However, it did not consider some of the names above the nested class . Thus we realized that by searching for "i" as in italic, it was possible to retrieve all names on the website. Some names had a title "Chair" next to their name, so this was removed after printing the full list of the names. We retrieved data about all the names using the Openalex API, and saved it to a authors.txt. There were duplicates in the results , but we chose to include everyone except if they had no results. In total we scraped results for 3902 people.


In [1]:
import networkx as nx
from collections import defaultdict
from itertools import combinations
from bs4 import BeautifulSoup  
import json
import pandas as pd
import concurrent.futures 
import requests
import math
import csv
from collections import Counter
import ast
import netwulf as nw
import matplotlib.pyplot as plt

In [ ]:
LINK = "https://ic2s2-2023.org/program"
r = requests.get(LINK) 
soup = BeautifulSoup(r.content) 
parallel_talks = soup.find_all('i')
pep=[]
for people in parallel_talks:
    people_list=people.text.split(",")
    for i in range(len(people_list)):
        pep.append(people_list[i].replace("Chair: ","").strip())
res=[]
[res.append(x) for x in pep if x not in res]
count=0
authors=[]
file1 = open('authors.txt', mode='a', encoding='utf-8')
for author in res:
    api_string="https://api.openalex.org/autocomplete/authors?q="
    temp_name=author.split(" ")
    api_string+=temp_name[0]+"%20"
    temp_name.pop(0)
    for last_name in temp_name:
        api_string+=last_name
    r = requests.get(api_string)
    soup = BeautifulSoup(r.content) 
    meta_data=''.join(str(e) for e in soup.text)
    meta_data=json.loads(meta_data)
    if(len(meta_data["results"])>0):
        for individual in meta_data["results"]:
            authors.append([individual["id"],individual["display_name"],individual["external_id"],individual["works_count"],individual["hint"]])
            temp=str(individual["id"])+","+str(individual["display_name"])+","+str(individual["external_id"])+","+str(individual["works_count"])+","+str(individual["hint"])+""
            file1.write(temp+"\n")
print(len(res))
print(len(authors))

# Part 2: Ready Made vs Custom Made Data

**2.1** The pros of the custom made data, as used in Centola's study, is that he is able to setup the experiment in a very specific way so it is useful for what he is researching. The cons is that there may be some significant bias present, since he is testing for a hypothesis. The results may not be representative of the real world in case the participants know what is going on. Furthermore, creating custom-made data can be time-consuming and resource-intensive.
The pros of ready-made data in Nicolaides' study is that he is able to gather a much larger data set at a lower cost. The cons associated with this data is that the purpose of the data may be entirely different, thus the data will not always be of use for all research purposes, e.g. lacking consideration for confounders or other meta data.

**2.2** The interpretation of the results in each study can be influenced by the aforementioned pros and cons. Fx. bias has to be considered to a large extent for custom-made data, since as a researcher you collect the data for a hypothesis you have. The confounders may not be considered when ready-made data is created by governments or large corporations to the same extent as custom-made data, thus extra precautions have to be made. In Centola's experiment, the controlled nature of the custom-made data allows for more confident conclusions about causal relationships, while in Nicolaides's study, using ready-made data can provide insights into real-world phenomena and population trends, but you have to be careful in generalizing findings and interpreting results considering the limitations of such datasets.

# Part 3: Gathering Research Articles using the OpenAlex API

In [2]:
f=open("authors.txt","r",encoding="utf-8")
url="https://api.openalex.org/works?per-page=200&filter=author.id:"
abstracts=[]
papers=[]
counters=[]
count=0
urls=[]
for line in f:
    temp=line.split(",")
    if(temp[3]!=None):
        if(int(temp[3])>5 and int(temp[3])<5000):
            if(int(temp[3])<201):
                temp_url=url+'"'+temp[0]+'"'
                urls.append(temp_url)
            else:
                pages=math.ceil(int(temp[3])/200)
                for i in range(pages):
                    page=i+1
                    url_max="https://api.openalex.org/works?page="+str(page)+"&per-page=200&filter=author.id:"+temp[0]+'"'
                    urls.append(url_max)
def process_data(url):
    r=requests.get(url).json()
    for abstract in r["results"]:
        if(int(abstract["cited_by_count"])>10 and len(abstract["authorships"])<10):
            focuses=["Sociology","Psychology","Economics","Political Science"]
            quantitative_disciplines=["Mathematics","Physics","Computer science"]
            focus=False
            quantitative=False
            for concept in abstract["concepts"]:
                if(int(concept["level"])==0):
                    if(concept["display_name"] in focuses):
                        focus=True
                    if(concept["display_name"] in quantitative_disciplines):
                        quantitative=True
            author_ids=[]
            for co_author in abstract["authorships"]:
                author_ids.append(co_author["author"]["id"])
            if(focus==True and quantitative==True):
                papers.append([abstract["id"],abstract["publication_year"],abstract["cited_by_count"],author_ids])
                abstracts.append([abstract["id"],abstract["title"],abstract["abstract_inverted_index"]])
    counters.append(["1"])
    print(len(counters))

with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    executor.map(process_data, urls)          
abstracts_df=pd.DataFrame(data=abstracts)
papers_df=pd.DataFrame(data=papers)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [2]:
abstracts_df=pd.read_csv("abstract.csv")
papers_df=pd.read_csv("papers.csv")

papers = []
#id,publication_year,cited_by_count,author_ids
with open("T_final_papers.csv","r") as f:
    csv_reader=csv.reader(f)
    for row in csv_reader:
        last_column_as_list = ast.literal_eval(row[-1])
        papers.append(row[:-1] + [last_column_as_list])
authors = []
unique_authors = []
#cited_by_count,country_code,display_name,h_index,id,works_api_url,works_count,topic
with open("T_final_authors.csv",encoding= "utf8") as f:
    csv_reader=csv.reader(f)
    for row in csv_reader:
        authors.append(row)
        unique_authors.append(row[4])
print(unique_authors[0])

id


**Efficiency in code**: \
Considering the extensiveness of the dataset we work with, we had to implement some strategies for code efficiency. We first retrived the data using the request.get() function. We then looped through all the authors to make sure to remove any duplicates before collecting all unique authors in a txt file. We also made sure to only include authors with a total work count betweenn 5 and 5,000. The most computational-heavy feature was extrating all the works from each author. To speed up this process, we applied multithreading techniques. \
\
**Filtering Criteria and Dataset Relevance**: \
Setting thresholds for authors' total works, citation counts, and relevance to specific fields ensures dataset quality. It includes important authors and influential works while filtering out less significant ones. However, this might exclude things like emerging researchers or niche topics. Regarding the dataset that we have compiled, we could miss out on interdisciplinary research or new methodologies due to emphasis on established topics. Overemphasizing on popular fields could also skew the research focus. Some aspects of Computational Social Science research could be over- or underrepresented as a result of this.

# Part 4: The Network of Computational Social Scientists

In [3]:
       
def make_pairs(temp_list):
    temp_list=list(temp_list)
    temp_list.sort()
    pairs=list(combinations(temp_list,2))
    return pairs
pairs=[]
for i in range(len(papers)):
    if(len(papers[i][3])>1):
        for pair in make_pairs(papers[i][3]):
            if(pair[1] in unique_authors and pair[0] in unique_authors):
                pairs.append(pair)  
def count_author_pairs(author_list):
    unique_pairs = {}
    for author_pair in author_list:
        sorted_author_pair = sorted(author_pair)
        sorted_author_pair_tuple = tuple(sorted_author_pair)
        if sorted_author_pair_tuple in unique_pairs:
            unique_pairs[sorted_author_pair_tuple] += 1
        else:
            unique_pairs[sorted_author_pair_tuple] = 1
    unique_author_list = [[list(author_pair), count] for author_pair, count in unique_pairs.items()]
    return unique_author_list
pairs=count_author_pairs(pairs)
for i in range(10):
    print(pairs[i])


[['https://openalex.org/A5003777693', 'https://openalex.org/A5023007469'], 4]
[['https://openalex.org/A5003777693', 'https://openalex.org/A5024505700'], 13]
[['https://openalex.org/A5003777693', 'https://openalex.org/A5044898565'], 11]
[['https://openalex.org/A5003777693', 'https://openalex.org/A5071524745'], 4]
[['https://openalex.org/A5003777693', 'https://openalex.org/A5091251187'], 6]
[['https://openalex.org/A5023007469', 'https://openalex.org/A5024505700'], 5]
[['https://openalex.org/A5023007469', 'https://openalex.org/A5044898565'], 7]
[['https://openalex.org/A5023007469', 'https://openalex.org/A5071524745'], 8]
[['https://openalex.org/A5023007469', 'https://openalex.org/A5091251187'], 4]
[['https://openalex.org/A5024505700', 'https://openalex.org/A5044898565'], 34]


In [4]:
G = nx.Graph()
count = 0
for pair in pairs:
    G.add_edge(pair[0][0],pair[0][1],attr=pair[1])
    count += 1
    print(str(count) + " : "+str(len(pairs)))



1 : 54304
2 : 54304
3 : 54304
4 : 54304
5 : 54304
6 : 54304
7 : 54304
8 : 54304
9 : 54304
10 : 54304
11 : 54304
12 : 54304
13 : 54304
14 : 54304
15 : 54304
16 : 54304
17 : 54304
18 : 54304
19 : 54304
20 : 54304
21 : 54304
22 : 54304
23 : 54304
24 : 54304
25 : 54304
26 : 54304
27 : 54304
28 : 54304
29 : 54304
30 : 54304
31 : 54304
32 : 54304
33 : 54304
34 : 54304
35 : 54304
36 : 54304
37 : 54304
38 : 54304
39 : 54304
40 : 54304
41 : 54304
42 : 54304
43 : 54304
44 : 54304
45 : 54304
46 : 54304
47 : 54304
48 : 54304
49 : 54304
50 : 54304
51 : 54304
52 : 54304
53 : 54304
54 : 54304
55 : 54304
56 : 54304
57 : 54304
58 : 54304
59 : 54304
60 : 54304
61 : 54304
62 : 54304
63 : 54304
64 : 54304
65 : 54304
66 : 54304
67 : 54304
68 : 54304
69 : 54304
70 : 54304
71 : 54304
72 : 54304
73 : 54304
74 : 54304
75 : 54304
76 : 54304
77 : 54304
78 : 54304
79 : 54304
80 : 54304
81 : 54304
82 : 54304
83 : 54304
84 : 54304
85 : 54304
86 : 54304
87 : 54304
88 : 54304
89 : 54304
90 : 54304
91 : 54304
92 : 543

In [5]:
print(G.number_of_nodes())
print(G.number_of_edges())
attributes = []
no_works = []
count = 0

for author in unique_authors:
    count += 1
    attribute = []
    for data in authors:
        if data[4] == author:
            #cited_by_count,country_code,display_name,h_index,id,works_api_url,works_count,topic
            attribute.append([data[4],data[2],data[1],data[0]])
    works_count = 0
    year = 3000
    print(count)
    for paper in papers:        
        for id in paper[3]:           
            if id == author:
                works_count += int(paper[2])
                if int(year) > int(paper[1]):
                    year = paper[1]
    if int(year) == 3000:
        no_works.append(author)
    attribute.append([year,works_count])
    attributes.append(attribute)


    

14196
54304
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274


In [92]:


count = 0
for node in G.nodes():
    count +=1
    print(count)
    for attribute in attributes:
        if(node == attribute[0][0]):
            color = "gray"
            if(attribute[0][2] == "US"):
                color = "yellow"
            if(attribute[0][2] == "CN"):
                color = "purple"
            if(attribute[0][2] == "GB"):
                color = "green"    
            if(attribute[0][2] == "IT"):
                color = "magenta"
            if(attribute[0][2] == "NL"):
                color = "orange"
            if(attribute[0][2] == "DE"):
                color = "blue"    
            node_attributes = {"id" : node, "display_name" : attribute[0][1], "country_code" : attribute[0][2], "cited_by_count" : attribute[0][3], "first_year_published" : attribute[1][0], "works_count" : attribute[1][1], "group" : color}
            G.nodes[node].update(node_attributes)
print(G.nodes["https://openalex.org/A5067809445"])


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [94]:
#nw.interactive.visualize(G) 
nw.visualize(G)

({'xlim': [0, 903],
  'ylim': [0, 903],
  'linkColor': '#7c7c7c',
  'linkAlpha': 0.44245905126568813,
  'nodeStrokeColor': '#555555',
  'nodeStrokeWidth': 0.32673899170389276,
  'links': [{'source': 'https://openalex.org/A5003777693',
    'target': 'https://openalex.org/A5023007469',
    'width': 0.12,
    'weight': 1},
   {'source': 'https://openalex.org/A5003777693',
    'target': 'https://openalex.org/A5024505700',
    'width': 0.12,
    'weight': 1},
   {'source': 'https://openalex.org/A5003777693',
    'target': 'https://openalex.org/A5044898565',
    'width': 0.12,
    'weight': 1},
   {'source': 'https://openalex.org/A5003777693',
    'target': 'https://openalex.org/A5071524745',
    'width': 0.12,
    'weight': 1},
   {'source': 'https://openalex.org/A5003777693',
    'target': 'https://openalex.org/A5091251187',
    'width': 0.12,
    'weight': 1},
   {'source': 'https://openalex.org/A5003777693',
    'target': 'https://openalex.org/A5015100237',
    'width': 0.12,
    'weight